In [8]:
import torch
print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Running on GPU:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU")

CUDA Available: True
Running on GPU: Quadro P2000


In [10]:
import os
import shutil
import random

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

random.seed(42)
DATASET_ROOT = r"A:\PyTorch\Project\140k_Real_Fake_Faces\real_vs_fake\real-vs-fake"

In [12]:
# =========================================
# ✓ SWINV2 TINY — FULL WORKING VERSION
# =========================================

# Basic imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import os
import timm


# Dataset class
class DeepfakeDataset(Dataset):
    def __init__(self, root, split="train", transform=None):
        self.root = os.path.join(root, split)
        self.transform = transform
        self.samples = []

        for label_name, label_value in [("real", 0), ("fake", 1)]:
            folder = os.path.join(self.root, label_name)
            for filename in os.listdir(folder):
                fpath = os.path.join(folder, filename)
                if os.path.isfile(fpath):
                    self.samples.append((fpath, label_value))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor([label], dtype=torch.float32)


# =========================================
# ✓ TRANSFORMS (SwinV2 uses 256x256)
# =========================================
train_transform = T.Compose([
    T.Resize((280, 280)),
    T.RandomResizedCrop(256, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(0.5),
    T.RandomApply([T.ColorJitter(0.3, 0.3, 0.2, 0.1)], p=0.7),
    T.RandomApply([T.GaussianBlur(kernel_size=3)], p=0.3),
    T.RandomGrayscale(p=0.02),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3),
])

val_transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3),
])


# =========================================
# ✓ DATALOADERS
# =========================================
train_ds = DeepfakeDataset(DATASET_ROOT, "train", train_transform)
val_ds   = DeepfakeDataset(DATASET_ROOT, "val", val_transform)
test_ds  = DeepfakeDataset(DATASET_ROOT, "test", val_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

In [13]:
# =========================================
# ✓ FIXED SWINV2 TINY MODEL
# =========================================
def get_swin_model(dropout=0.3):
    # Load SwinV2 Tiny WITHOUT classifier
    model = timm.create_model(
        "swinv2_tiny_window8_256.ms_in1k",
        pretrained=True,
        num_classes=0,
        features_only=False
    )

    # -------------------------------------------------
    # 🔍 AUTO-DETECT FEATURE SIZE
    # -------------------------------------------------
    model.eval()
    with torch.no_grad():
        dummy = torch.randn(1, 3, 256, 256)
        feats = model.forward_features(dummy)  # raw features
        # feats shape likely: (1, C, H, W) OR (1, N, C)
        if feats.ndim == 4:   # (B, C, H, W)
            feats = nn.AdaptiveAvgPool2d(1)(feats).flatten(1)
        elif feats.ndim == 3: # (B, N, C)
            feats = feats.mean(dim=1)
        feat_dim = feats.shape[1]  # final feature dimension

    # -------------------------------------------------
    # 🔵 BUILD CLASSIFIER HEAD WITH CORRECT DIM
    # -------------------------------------------------
    model.head = nn.Sequential(
        nn.Dropout(dropout),
        nn.Linear(feat_dim, 1)
    )

    # Wrap forward() to include our head
    def forward(self, x):
        x = self.forward_features(x)
        if x.ndim == 4:
            x = nn.AdaptiveAvgPool2d(1)(x).flatten(1)
        elif x.ndim == 3:
            x = x.mean(dim=1)
        x = self.head(x)
        return x

    model.forward = forward.__get__(model, model.__class__)
    return model


device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_swin_model().to(device)

print("✔ SwinV2 Tiny fully initialized!")


# =========================================
# ✓ LOSS, OPTIMIZER, SCHEDULER
# =========================================
criterion = nn.BCEWithLogitsLoss()

head_params = [p for n, p in model.named_parameters() if "head" in n]
backbone_params = [p for n, p in model.named_parameters() if "head" not in n]

optimizer = torch.optim.AdamW([
    {"params": backbone_params, "lr": 1e-5, "weight_decay": 1e-4},
    {"params": head_params,     "lr": 1e-4, "weight_decay": 1e-3},
])

scaler = torch.cuda.amp.GradScaler()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=2,
)

print("✔ Optimizer & Scheduler ready!")

a:\PyTorch\Environment\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub\models--timm--swinv2_tiny_window8_256.ms_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✔ SwinV2 Tiny fully initialized!
✔ Optimizer & Scheduler ready!


C:\Users\ahmed\AppData\Local\Temp\ipykernel_1348\957326209.py:68: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [14]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    from tqdm import tqdm
    loop = tqdm(train_loader, leave=False)

    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

        preds = (torch.sigmoid(outputs) >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_description(f"loss: {loss.item():.4f}")

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100
    return epoch_loss, epoch_acc

In [15]:
# ===============================
#   CHECKPOINT SAVE / LOAD
# ===============================

def save_checkpoint(epoch, model, optimizer, scaler, scheduler, best_val_acc, filename):
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict(),
        "scheduler_state": scheduler.state_dict(),
        "best_val_acc": best_val_acc,
        "model_name": model.__class__.__name__,      # ⭐ NEW — protects against wrong model loading
        "timm_name": getattr(model, "default_cfg", {}).get("architecture", None)  # bonus
    }

    torch.save(checkpoint, filename)
    print(f"💾 Saved checkpoint to: {filename}")


def load_checkpoint(path, model, optimizer, scaler, scheduler):
    print(f"🔄 Loading checkpoint: {path}")

    checkpoint = torch.load(path, map_location=device)

    # ===============================
    #   MODEL MATCHING SAFETY CHECK
    # ===============================
    saved_name = checkpoint.get("model_name", None)
    current_name = model.__class__.__name__

    if saved_name is not None and saved_name != current_name:
        print(f"⚠️ WARNING: Checkpoint was trained on: {saved_name}")
        print(f"⚠️ Current model is: {current_name}")
        print("⚠️ Loading anyway, but ensure architectures are compatible.")

    # Load states
    model.load_state_dict(checkpoint["model_state"], strict=True)
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    scaler.load_state_dict(checkpoint["scaler_state"])
    scheduler.load_state_dict(checkpoint["scheduler_state"])

    start_epoch = checkpoint["epoch"] + 1
    best_val_acc = checkpoint["best_val_acc"]

    print(f"✔ Resumed from epoch {start_epoch}")
    print(f"✔ Previous best val acc: {best_val_acc:.2f}%")

    return start_epoch, best_val_acc

In [16]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)

            preds = (torch.sigmoid(outputs) >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = correct / total * 100
    return epoch_loss, epoch_acc

In [ ]:
CHECKPOINT_DIR = r"A:\PyTorch"  # 🔵 NEW
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ---- OPTIONAL: choose checkpoint to resume ----
resume = False
checkpoint_path = f"{CHECKPOINT_DIR}/checkpoint_epoch_8.pth"

if resume:
    start_epoch, best_val_acc = load_checkpoint(
        checkpoint_path, model, optimizer, scaler, scheduler
    )
else:
    start_epoch = 0
    best_val_acc = 0.0

epochs = 10   # 🔵 Swin learns slower → use 10 epochs
patience = 3
no_improve = 0

# ---- TRAIN LOOP ----
for epoch in range(start_epoch, epochs):
    print(f"\n🔵 Epoch {epoch+1}/{epochs}")

    train_loss, train_acc = train_one_epoch(
    model, train_loader, criterion, optimizer, device
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")

    # Scheduler step
    scheduler.step(val_acc)

    # ---- BEST MODEL ----
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        no_improve = 0
        save_checkpoint(
            epoch, model, optimizer, scaler, scheduler,
            best_val_acc, f"{CHECKPOINT_DIR}/best_model.pth"
        )
        print("🏆 Best model updated!")
    else:
        no_improve += 1

    # ---- EARLY STOPPING ----
    if no_improve >= patience:
        print("⛔ Early stopping triggered!")
        break

    # ---- EPOCH CHECKPOINT ----
    save_checkpoint(
        epoch, model, optimizer, scaler, scheduler,
        best_val_acc,
        f"{CHECKPOINT_DIR}/checkpoint_epoch_{epoch+1}.pth"
    )


🔵 Epoch 1/10


loss: 0.6966:   0%|          | 4/3125 [02:17<30:38:25, 35.34s/it]

In [17]:
# Cell 9 — evaluation & single-image prediction (NO NUMPY)
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import torch
from PIL import Image

# Load best model
best_model_path = r"A:\PyTorch\checkpoint_epoch_8.pth"
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint["model_state"])
model.to(device).eval()

# -------------------------------
#   Get predictions + probabilities
# -------------------------------
def get_preds_probs(model, loader):
    all_labels = []
    all_probs = []

    model.eval()
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)

            outputs = model(imgs)
            probs = torch.sigmoid(outputs).squeeze().cpu()   # torch tensor

            all_probs.extend(probs.tolist())                  # pure Python list
            all_labels.extend(labels.squeeze().tolist())      # pure Python list

    return all_labels, all_probs


# Run evaluation
y_true, y_probs = get_preds_probs(model, test_loader)

# Convert to hard predictions
y_pred = [1 if p >= 0.5 else 0 for p in y_probs]

# -------------------------------
#   Metrics
# -------------------------------
accuracy = sum([yt == yp for yt, yp in zip(y_true, y_pred)]) / len(y_true) * 100
print("Test Accuracy:", accuracy)

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["real", "fake"]))

# ROC-AUC needs Python lists (not numpy)
print("ROC AUC:", roc_auc_score(y_true, y_probs))

# -------------------------------
#   Single image prediction
# -------------------------------
def predict_image(model, image_path, transform):
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(x)
        p = torch.sigmoid(out).item()

    return p

Test Accuracy: 99.97
Confusion Matrix:
 [[9999    1]
 [   5 9995]]
              precision    recall  f1-score   support

        real       1.00      1.00      1.00     10000
        fake       1.00      1.00      1.00     10000

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

ROC AUC: 0.99999953
